In [11]:
# manipulation des données
import numpy as np
import pandas as pd

# matplotlib et seaborn pour les représentations graphiques
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# sklearn preprocessing pour le traiter les variables catégorielles
from sklearn.preprocessing import LabelEncoder

# Gestion du système de fichiers
import os

# Suppression des alertes
import warnings
warnings.filterwarnings('ignore')

In [12]:
df = pd.read_csv("../Data/2.sample_dataset.csv")
df = df.dropna(subset=['words'])

In [13]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import gensim

In [14]:
# Création et entraînement du modèle Word2Vec
w2v_size=300
w2v_window=5
w2v_min_count=1
w2v_epochs=100

sentences = df['words'].to_list()

print("Build & train Word2Vec model ...")
w2v_model = gensim.models.Word2Vec(min_count=w2v_min_count, window=w2v_window,
                                                vector_size=w2v_size,
                                                seed=42,
                                                workers=1)
#                                                workers=multiprocessing.cpu_count())
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

Build & train Word2Vec model ...
Vocabulary size: 61
Word2Vec trained


In [15]:
# Préparation des sentences (tokenization)
maxlen = 24 # adapt to length of sentences

print("Fit Tokenizer ...")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
x_sentences = pad_sequences(tokenizer.texts_to_sequences(sentences),
                                                     maxlen=maxlen,
                                                     padding='post') 
                                                   
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

Fit Tokenizer ...
Number of unique words: 292434


In [16]:
print("Create Embedding matrix ...")
w2v_size = 300
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i=0
j=0
    
for word, idx in word_index.items():
    i +=1
    if word in w2v_words:
        j +=1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]
            
word_rate = np.round(j/i,4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))

Create Embedding matrix ...
Word embedding rate :  0.0001
Embedding matrix: (292434, 300)


In [17]:
from sklearn.model_selection import train_test_split

labels = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(x_sentences, labels, test_size=0.2, random_state=42)

In [18]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=vocab_size, output_dim=w2v_size, input_length=maxlen, weights=[embedding_matrix], trainable=False))
cnn_model.add(Conv1D(128, 5, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))

In [19]:
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
import mlflow
import mlflow.keras

mlflow.log_param("w2v_size", w2v_size)
mlflow.log_param("w2v_window", w2v_window)
mlflow.log_param("w2v_min_count", w2v_min_count)
mlflow.log_param("w2v_epochs", w2v_epochs)
mlflow.log_param("maxlen", maxlen)

mlflow.keras.log_model(cnn_model, "model")

mlflow.set_experiment("CNN")

with mlflow.start_run(run_name=f"Word2Vec_CNN"):
    cnn_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

    loss, accuracy = cnn_model.evaluate(X_test, y_test)
    mlflow.log_metric("loss", loss)
    mlflow.log_metric("accuracy", accuracy)

Epoch 1/5
24888/24888 [==============================] - 160s 6ms/step - loss: 0.6932 - accuracy: 0.5003 - val_loss: 0.6933 - val_accuracy: 0.4998
Epoch 2/5
24888/24888 [==============================] - 154s 6ms/step - loss: 0.6931 - accuracy: 0.4997 - val_loss: 0.6934 - val_accuracy: 0.5025
Epoch 3/5
24888/24888 [==============================] - 154s 6ms/step - loss: 0.6930 - accuracy: 0.5009 - val_loss: 0.6929 - val_accuracy: 0.4989
Epoch 4/5
24888/24888 [==============================] - 156s 6ms/step - loss: 0.6930 - accuracy: 0.5002 - val_loss: 0.6929 - val_accuracy: 0.4988
Epoch 5/5
24888/24888 [==============================] - 151s 6ms/step - loss: 0.6930 - accuracy: 0.4996 - val_loss: 0.6929 - val_accuracy: 0.4990


In [21]:
loss, accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')

6222/6222 [==============================] - 15s 2ms/step - loss: 0.6929 - accuracy: 0.4990
Model Accuracy: 49.90%
